### Base

In [1]:
!pip install ultralytics
!pip install roboflow
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.4/715.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstall

Looking in indexes: https://download.pytorch.org/whl/cu121


In [2]:
import os
import random
import argparse
from collections import deque

import cv2
import tqdm
import numpy as np
import pandas as pd
from PIL import Image
from numpy import random
import matplotlib.pyplot as plt
from IPython.display import display, Image, clear_output

import torch
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset

from ultralytics import YOLO # yolov8

import warnings
warnings.filterwarnings("ignore")

In [3]:
devices = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using {devices} device")

Using cuda device


In [4]:
!nvidia-smi

Tue Feb 20 21:47:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!yolo checks

Ultralytics YOLOv8.1.16 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.5/78.2 GB disk)

OS                  Linux-6.1.58+-x86_64-with-glibc2.35
Environment         Colab
Python              3.10.12
Install             pip
RAM                 12.67 GB
CPU                 Intel Xeon 2.20GHz
CUDA                12.1

matplotlib          ✅ 3.7.1>=3.3.0
opencv-python       ✅ 4.8.0.76>=4.6.0
pillow              ✅ 9.4.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.11.4>=1.4.1
torch               ✅ 2.1.0+cu121>=1.8.0
torchvision         ✅ 0.16.0+cu121>=0.9.0
tqdm                ✅ 4.66.2>=4.64.0
psutil              ✅ 5.9.5
py-cpuinfo          ✅ 9.0.0
thop                ✅ 0.1.1-2209072238>=0.1.1
pandas              ✅ 1.5.3>=1.1.4
seaborn             ✅ 0.13.1>=0.11.0


### Load OpenSource Dataset

In [6]:
from roboflow import Roboflow

rf = Roboflow(api_key="HXl0GPgbhL73iBM3pgok")

In [7]:

# Link: https://universe.roboflow.com/firedetection-cqgz2/firedetectyolov7/dataset/2
fire1_project = rf.workspace("firedetection-cqgz2").project("firedetectyolov7")
fire1_dataset = fire1_project.version(2).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.16, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to firedetectyolov7-2 in yolov8:: 100%|██████████| 15378/15378 [00:02<00:00, 5787.44it/s]


In [8]:
# Link: https://universe.roboflow.com/agostino-abbatecola-52ty4/veryfiresmokedetection/dataset/3
fire2_project = rf.workspace("agostino-abbatecola-52ty4").project("veryfiresmokedetection")
fire2_dataset = fire2_project.version(3).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.16, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Veryfiresmokedetection-3 in yolov8:: 100%|██████████| 14790/14790 [00:02<00:00, 6345.09it/s]


In [9]:
# Link: https://universe.roboflow.com/atlantic-technological-university/wildfire-v2/dataset/5
fire3_project = rf.workspace("atlantic-technological-university").project("wildfire-v2")
fire3_dataset = fire3_project.version(5).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.16, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Wildfire-V2-5 in yolov8:: 100%|██████████| 268954/268954 [01:40<00:00, 2684.08it/s]


In [10]:
# Link: https://universe.roboflow.com/spyrobot/fire-smoke-and-human-detector/dataset/31
fire4_project = rf.workspace("spyrobot").project("fire-smoke-and-human-detector")
fire4_dataset = fire4_project.version(31).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.16, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Fire-Smoke-and-Human-Detector-31 in yolov8:: 100%|██████████| 51514/51514 [00:17<00:00, 3030.18it/s]


### Training Process

In [12]:
# Train the model on fire4 dataset
!yolo task=detect mode=train model=yolov8m.pt imgsz=640 data=/content/Veryfiresmokedetection-3/data.yaml epochs=30 batch=16 name=data4 patience=10 plots=True val=True verbose=True pretrained=/content/best.pt

Transferred 469/475 items from pretrained weights
Ultralytics YOLOv8.1.16 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/Veryfiresmokedetection-3/data.yaml, epochs=30, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=data42, exist_ok=False, pretrained=/content/best.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=F

### Test Process

In [ ]:
# Test the model on fire4 dataset
!yolo task=detect mode=val model=D:/Fire-Alarm-Detection/runs/detect/yolo8m-30epoch/weights/best.pt data={fire3_dataset.location}/data.yaml imgsz=640 verbose=True

Ultralytics YOLOv8.0.196  Python-3.11.7 torch-2.2.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 268 layers, 43608921 parameters, 0 gradients, 164.8 GFLOPs

val: Scanning D:\Fire-Alarm-Detection\Wildfire-V2-5\valid\labels...:   0%|          | 0/11205 [00:00<?, ?it/s]
val: Scanning D:\Fire-Alarm-Detection\Wildfire-V2-5\valid\labels... 41 images, 0 backgrounds, 0 corrupt:   0%|          | 41/11205 [00:00<00:28, 396.63it/s]
val: Scanning D:\Fire-Alarm-Detection\Wildfire-V2-5\valid\labels... 113 images, 0 backgrounds, 0 corrupt:   1%|          | 113/11205 [00:00<00:19, 569.85it/s]
val: Scanning D:\Fire-Alarm-Detection\Wildfire-V2-5\valid\labels... 177 images, 0 backgrounds, 0 corrupt:   2%|▏         | 177/11205 [00:00<00:18, 599.05it/s]
val: Scanning D:\Fire-Alarm-Detection\Wildfire-V2-5\valid\labels... 241 images, 0 backgrounds, 0 corrupt:   2%|▏         | 241/11205 [00:00<00:17, 612.86it/s]
val: Scanning D:\Fire-Alarm-Detection\Wildfire-V2-5\valid\lab

In [ ]:
# Test predict on video
!yolo task=detect mode=predict model=D:/Fire-Alarm-Detection/runs/detect/yolo8m-30epoch/weights/best.pt source=D:/Fire-Alarm-Detection/check/video_data/train_videos/fire_and_smoke.mp4 imgsz=640

Ultralytics YOLOv8.0.196  Python-3.11.7 torch-2.2.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs

video 1/1 (1/1474) D:\Fire-Alarm-Detection\check\video_data\train_videos\fire_and_smoke.mp4: 384x640 (no detections), 139.6ms
video 1/1 (2/1474) D:\Fire-Alarm-Detection\check\video_data\train_videos\fire_and_smoke.mp4: 384x640 (no detections), 19.1ms
video 1/1 (3/1474) D:\Fire-Alarm-Detection\check\video_data\train_videos\fire_and_smoke.mp4: 384x640 (no detections), 19.0ms
video 1/1 (4/1474) D:\Fire-Alarm-Detection\check\video_data\train_videos\fire_and_smoke.mp4: 384x640 (no detections), 18.4ms
video 1/1 (5/1474) D:\Fire-Alarm-Detection\check\video_data\train_videos\fire_and_smoke.mp4: 384x640 1 fire, 17.5ms
video 1/1 (6/1474) D:\Fire-Alarm-Detection\check\video_data\train_videos\fire_and_smoke.mp4: 384x640 (no detections), 18.1ms
video 1/1 (7/1474) D:\Fire-Alarm-Detection\check\video_data\train